<h1> <span style ="color:blue"> <strong> ETL PROJECT </strong> </span> </h1>

<em>Team Name: All About that Database</em>
<br>
<em>Team Members: Marissa, Star, Heather, Valentina, Claudia</em>

<h3> Contents: </h3>
    Analysis Question <br>
    Step by Step: Extract, Transform, and Load <br>
    Query Issue and Discovery <br>
    What we would've done differently <br>

<p><span style="color:blue"><h3>Analysis Question: </h3></span>Are NBA players that are awarded "Player of the Week" have higher salaries? If so, in which seasons are their salaries higher? .</p>
<br>
We looked at NBA data, specifically focusing on players who got player of the week from 1985-86 to 2017-18 season and seeing their stats. We got players stats & salaries info from data.world and player of the week info from Kaggle. Initially, we wanted to look at all of the stats for players who got player of the week in one comprehensive table but ran into issues regarding duplicates so we decided to focus on two fields (salaries & draft year) for simplicity’s sake for this project. 

<span style = "color:gray"> <em><strong>Kaggle.com</strong>
<br>
Datasource:  NBA.com and Basketball Reference 
<br>
File Name:  NBA_player_of_the_week 
<br>
<br>
<strong>Data.world</strong>
<br>
Datasource:  Basketball Reference 
<br>
File Name: salaries_1985to2018
<br>
File Name: players</em></span>

## Extract, Transform, Load: Step by Step   
<h3> <span style ="color:green">Retrieving the Data & Loading into Google Cloud Storage</span></h3>
<br>

<h4> Retrieving </h4>
<p>
<ul style="list-style-type:disc">
    <li>Retrieved the data from each source found on Kaggle.com and Data.world</li>
<li>Downloaded files as .csv to Local Drive to be pushed into GitHub</li><br>
   link: https://github.com/Rotas20/Data-Bass/blob/master/Resources/salaries_1985to2018.csv <br>
   link: https://github.com/Rotas20/Data-Bass/blob/master/Resources/players.csv <br>
   link: https://github.com/Rotas20/Data-Bass/blob/master/Resources/NBA_player_of_the_week.csv <br>
    <br>
<h4>Loading </h4>
<p>
<ul style="list-style-type:disc">
     <li>We created a New Project in Google Cloud. Our reasoning for choosing GC was because of the functionality of the project. This program would allow for us to do the ETL data process from start to finish - while allowing for multiple collaborators.</li>
     <li>To be able to connect to the Google Cloud Storage system, we had to create a storage bucket within the platform. We went into the "storage" platform and created a bucket for "NBA" and a folder within the bucket to hold our resources folder. </li>
     <li>All files were uploaded to our GCS resources folder</li>
     <li>Once the loading was complete, we navigated back into the BigQuery editor to create the new table schema "NBA" and prepared it for table creation. </li>
      </ul></p>



<h3> <span style ="color:magenta">Creating, Cleaning & Joining Data Tables</span></h3>
<p>We needed to consolidate several files to transform our dataset. We chose to create views for the first two joins (Player_join & POTW) because it hides the complexity of the code from the end-user (simulates real-life scenarios for requests from other departments at work) and is suitable for layering multiple queries while saving space.   </p>
<h4> Table Creation </h4>

<p>
<ul style="list-style-type:disc">
    <li>The tables were created using the "create table" function in BigQuery.</li>
    <li>Due to the data being stored in the GCS, we were able to upload the CSV files directly in the UI and selected for Google to identify the schema.</li>
    <li>For advanced settings, we identified that there was a header row included in the file and only accounted for 10 errors at maximum.</li>
    </ul>
    </p>
<h4>Data Cleaning</h4>
<p>
<ul style="list-style-type:disc">
    <li>There wasn't much to clean as our data was fairly clean.</li>
    <li>Before joining tables together, we decided what fields we needed to keep and which we didn't.</li>
    </ul>
    </p>

<h4>Joining Tables</h4>
<p>
<ul style="list-style-type:disc">
    <li>In order for the salaries information to be included in the master players table - we had to join the two together. We joined the two tables on "_id" and "player_id" as those were the same unique fields. 
        <br><span style='color:darkblue'> Player Data and Salaries Table </span><br><code>SELECT p.name, p.career_FG_, p.birthPlace, p.position, p.college, p.career_FG3_, p.career_PTS, p.career_WS, p.career_PER, p.career_FT_, p.career_AST, p.career_TRB, p.highSchool, s.team, s.season, s.salary, 
FROM `all-about-that-database.nba.sal_players` as p
JOIN `all-about-that-database.nba.salaries` as s
        ON p._id = s.player_id</code> <br>       
    <li>Once we created a full players master data profile, we had to join the "player of the week" table in order to finish our master data file. We did this by joining the data on "name" and "player_name."</li>
<span style='color:darkblue'> Player of the Week View</span>
<br><code>SELECT Player, Date, Age, Weight, Height, Pre_draft_Team, Draft_Year, Conference,
FROM `all-about-that-database.nba.player_otw`
</code>
    <span style='color:darkblue'> Final (UPDATED) Code</span><br>
    <code>SELECT v.*, FORMAT("%.f", salary)
FROM (SELECT name, avg(salary) salary FROM `all-about-that-database.nba.player_join` group by name) as j
JOIN (SELECT Player, min(Draft_Year) draft_year FROM `all-about-that-database.nba.potw_view` group by Player) as v
ON j.name = v.Player</code>
<li>After doing the above join - we noticed upon further data cleaning in Jupyter notebook, that the join created duplicative rows and inturn created an incorrect output. (File name: nba_final_data_output.csv) **See the exploration below** </li><br>
    </ul>
    </p>



<h2> <span style="color:blue"> Query Issue & Discovery </span> </h2> 

<p><span style = "color:purple"> <strong> How we found the issue: </strong> 
    <br>
    After exporting the "final" data to a CSV file on our local drive, we read it into Jupyter notebook using pandas. We wanted to use Jupyter notebook to finish our data transformation due to easier maniuplation. After going through the below code, we identified that there was an issue when we joined "player_join" and "potw_view." Due to the fact that we did not originally use sub-queries to aggregate the data, it in turn created duplicative rows with the incorrect salary information and season year. *Identified below in the final kernel* </span></p>

In [40]:
#Importing our final table from GCP
import pandas as pd
csv = 'nba_final_data_output.csv'
data = pd.read_csv(csv)

In [41]:
data.columns

Index(['name', 'career_FG_', 'birthPlace', 'position', 'college',
       'career_FG3_', 'career_PTS', 'career_WS', 'career_PER', 'career_FT_',
       'career_AST', 'career_TRB', 'highSchool', 'team', 'season', 'salary',
       'Player', 'Date', 'Age', 'Weight', 'Height', 'Pre_draft_Team',
       'Draft_Year', 'Conference'],
      dtype='object')

In [42]:
#Dropping unnecessary columns
del data['name']
del data['Pre_draft_Team']
del data['birthPlace']
del data['career_TRB']
del data['career_AST']

In [43]:
data.columns

Index(['career_FG_', 'position', 'college', 'career_FG3_', 'career_PTS',
       'career_WS', 'career_PER', 'career_FT_', 'highSchool', 'team', 'season',
       'salary', 'Player', 'Date', 'Age', 'Weight', 'Height', 'Draft_Year',
       'Conference'],
      dtype='object')

In [44]:
cleaned_names = data.rename(columns={'Player': 'player','career_FG_' : 'field_goal','career_FG3_' : 'three_pts', 
                                      'career_FT_' : 'free_throw', 'highSchool': 'high_school', 'Date': 'player_of_week', 'Age': 'age', 
                                       'Weight' : 'weight', 'Height' : 'height', 'Draft_Year': 'draft_year',
                                       'Conference':'conference', 'career_PTS' : 'points', 'career_WS': 'win_share', 'career_PER': 'efficiency'})

In [45]:
cleaned_names.columns



Index(['field_goal', 'position', 'college', 'three_pts', 'points', 'win_share',
       'efficiency', 'free_throw', 'high_school', 'team', 'season', 'salary',
       'player', 'player_of_week', 'age', 'weight', 'height', 'draft_year',
       'conference'],
      dtype='object')

In [46]:
#Reorganizing the columns
organized_data = cleaned_names[["player_of_week", "season", "player", "salary", "team", "conference",
                                "position", "win_share", "efficiency", "points", "field_goal", 
                                "three_pts", "free_throw", "age", "weight", "height", 
                                "college", "high_school", "draft_year"]]
organized_data.head()

,player_of_week,season,player,salary,team,conference,position,win_share,efficiency,points,field_goal,three_pts,free_throw,age,weight,height,college,high_school,draft_year
0,23-Mar-86,1984-85,Adrian Dantley,515000,Utah Jazz,NaN,Small Forward,134.2,21.5,24.3,54.0,17.1,81.8,30,208,6'5,University of Notre Dame,"DeMatha Catholic in Hyattsville, Maryland",1976
1,18-Mar-84,1984-85,Adrian Dantley,515000,Utah Jazz,NaN,Small Forward,134.2,21.5,24.3,54.0,17.1,81.8,28,208,6'5,University of Notre Dame,"DeMatha Catholic in Hyattsville, Maryland",1976
2,18-Dec-83,1984-85,Adrian Dantley,515000,Utah Jazz,NaN,Small Forward,134.2,21.5,24.3,54.0,17.1,81.8,28,208,6'5,University of Notre Dame,"DeMatha Catholic in Hyattsville, Maryland",1976
3,19-Oct-80,1984-85,Adrian Dantley,515000,Utah Jazz,NaN,Small Forward,134.2,21.5,24.3,54.0,17.1,81.8,25,208,6'5,University of Notre Dame,"DeMatha Catholic in Hyattsville, Maryland",1976
4,2-Dec-79,1984-85,Adrian Dantley,515000,Utah Jazz,NaN,Small Forward,134.2,21.5,24.3,54.0,17.1,81.8,24,208,6'5,University of Notre Dame,"DeMatha Catholic in Hyattsville, Maryland",1976


In [47]:
#Filling the 'NaN' values as 'East' or 'West' based on teams' conference assignment
organized_data.loc[organized_data['team'] == 'Atlanta Hawks', 'conference'] = 'East'
organized_data.loc[organized_data['team'] == 'Boston Celtics', 'conference'] = 'East'
organized_data.loc[organized_data['team'] == 'Brooklyn Nets', 'conference'] = 'East'
organized_data.loc[organized_data['team'] == 'Charlotte Bobcats', 'conference'] = 'East'
organized_data.loc[organized_data['team'] == 'Charlotte Hornets', 'conference'] = 'East'
organized_data.loc[organized_data['team'] == 'Chicago Bulls', 'conference'] = 'East'
organized_data.loc[organized_data['team'] == 'Cleveland Cavaliers', 'conference'] = 'East'
organized_data.loc[organized_data['team'] == 'Detroit Pistons', 'conference'] = 'East'
organized_data.loc[organized_data['team'] == 'Indiana Pacers', 'conference'] = 'East'
organized_data.loc[organized_data['team'] == 'Miami Heat', 'conference'] = 'East'
organized_data.loc[organized_data['team'] == 'Milwaukee Bucks', 'conference'] = 'East'
organized_data.loc[organized_data['team'] == 'New Jersey Nets', 'conference'] = 'East'
organized_data.loc[organized_data['team'] == 'New York Knicks', 'conference'] = 'East'
organized_data.loc[organized_data['team'] == 'Orlando Magic', 'conference'] = 'East'
organized_data.loc[organized_data['team'] == 'Philadelphia 76ers', 'conference'] = 'East'
organized_data.loc[organized_data['team'] == 'Toronoto Raptors', 'conference'] = 'East'
organized_data.loc[organized_data['team'] == 'Washington Wizards', 'conference'] = 'East'
organized_data.loc[organized_data['team'] == 'Washington Bullets', 'conference'] = 'East'

In [48]:
organized_data.loc[organized_data['conference'].isnull(), 'conference'] = 'West'

In [49]:
organized_data.head()

,player_of_week,season,player,salary,team,conference,position,win_share,efficiency,points,field_goal,three_pts,free_throw,age,weight,height,college,high_school,draft_year
0,23-Mar-86,1984-85,Adrian Dantley,515000,Utah Jazz,West,Small Forward,134.2,21.5,24.3,54.0,17.1,81.8,30,208,6'5,University of Notre Dame,"DeMatha Catholic in Hyattsville, Maryland",1976
1,18-Mar-84,1984-85,Adrian Dantley,515000,Utah Jazz,West,Small Forward,134.2,21.5,24.3,54.0,17.1,81.8,28,208,6'5,University of Notre Dame,"DeMatha Catholic in Hyattsville, Maryland",1976
2,18-Dec-83,1984-85,Adrian Dantley,515000,Utah Jazz,West,Small Forward,134.2,21.5,24.3,54.0,17.1,81.8,28,208,6'5,University of Notre Dame,"DeMatha Catholic in Hyattsville, Maryland",1976
3,19-Oct-80,1984-85,Adrian Dantley,515000,Utah Jazz,West,Small Forward,134.2,21.5,24.3,54.0,17.1,81.8,25,208,6'5,University of Notre Dame,"DeMatha Catholic in Hyattsville, Maryland",1976
4,2-Dec-79,1984-85,Adrian Dantley,515000,Utah Jazz,West,Small Forward,134.2,21.5,24.3,54.0,17.1,81.8,24,208,6'5,University of Notre Dame,"DeMatha Catholic in Hyattsville, Maryland",1976
